## Initialize the environment

In [ ]:
from pathlib import Path
from datetime import datetime
import logging
import os
import sys

ENV = "jupytern"
os.environ['CS_ENV'] = "jupytern"
BASE_DIR = "/home/jupyter/CivilServant"
sys.path.append(BASE_DIR)

LOG_PATH = str(Path(BASE_DIR, "logs", "praw_messenger_%s.log" % ENV))
logging.basicConfig(filename=LOG_PATH, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

notebook_dir = os.getcwd()
os.chdir(BASE_DIR)
import reddit.connection
conn = reddit.connection.Connect(base_dir=BASE_DIR, env="jupytern")
r = conn.connect(use_db_keys=False)
os.chdir(notebook_dir)
import praw.errors


## Define the message template

In [ ]:
MESSAGE_SUBJECT = "Survey Notification"
MESSAGE_TEMPLATE = """Dear {user},
Please take the following survey: {url}

Pretending these are being passed into the form:

first_comment_age: {first_comment_age}
get_info_date: {get_info_date}
deleted: {deleted}
"""

## Add the ability to validate users

In [ ]:
def get_reddit_user(username):
    try:
        user_dict = r.get_redditor(username, fetch=True).json_dict
        logging.info("User %s found: %s" %(username, str(user_dict)))
        return user_dict
    except praw.errors.NotFound as e:
        logging.error("User not found: %s" % username)
    except Exception as e:
        logging.exception("Failed to retrieve user")

## Add the ability to send messages

In [ ]:
def send_reddit_message(**message_data):
    user = message_data["user"]
    message = MESSAGE_TEMPLATE.format(**message_data)
    try:
        logging.info("Sending a message to user %s with data %s" % (user, str(message_data)))
        response = r.send_message(user, MESSAGE_SUBJECT, message, raise_captcha_exception=True)
        if response["errors"]:
            logging.error("Error in response when sending a message to user %s: %s" % (user, str(response)))
        else:
            logging.info("Message successfully sent to user %s" % user)
    except praw.errors.InvalidCaptcha as e:
        logging.exception("Message sending requires a captcha")
        logging.error(e.response)
    except Exception as e:
        logging.exception("Failed to send message to %s" % user)

## Wrap the messenger function with anything survey-specific

In [2]:
def send_survey_message(**message_data):
    logging.info("first_comment_age: %s" % message_data["first_comment_age"])
    logging.info("url: %s" % message_data["url"])
    logging.info("get_info_data: %s" % message_data["get_info_date"])
    logging.info("deleted: %s" % message_data["deleted"])
    send_reddit_message(**message_data)

## Send a message to each valid reddit user

In [3]:
#users = ["this_user_does_not_exist", "epenn", "natematias"]
users = ["this_user_does_not_exist", "natematias"]


for username in users:
    logging.info("Attempting to a send a message to user %s" % username)
    user = get_reddit_user(username)
    if user:
        send_survey_message(
            user = username,
            first_comment_age = 100,
            get_info_date = str(datetime.utcnow()),
            deleted = False,
            url = "http://dummy.survey.url.com")

NameError: name 'logging' is not defined